# HW 7 Phonemes identifier using Neural Networks

In [15]:
import pandas as pd

# Read in Data
df = pd.read_csv('/Users/kevin/Desktop/Stats 760/Homework7/phoneme.data.txt')
df.head()

,row.names,x.1,x.2,x.3,x.4,x.5,x.6,x.7,x.8,x.9,...,x.249,x.250,x.251,x.252,x.253,x.254,x.255,x.256,g,speaker
0,1,9.85770,9.20711,9.81689,9.01692,9.05675,8.92518,11.28308,11.52980,10.79713,...,12.68076,11.20767,13.69394,13.72055,12.16628,12.92489,12.51195,9.75527,sh,train.dr1.mcpm0.sa1
1,2,13.23079,14.19189,15.34428,18.11737,19.53875,18.32726,17.34169,17.16861,19.63557,...,8.45714,8.77266,9.59717,8.45336,7.57730,5.38504,9.43063,8.59328,iy,train.dr1.mcpm0.sa1
2,3,10.81889,9.07615,9.77940,12.20135,12.59005,10.53364,8.54693,9.46049,11.96755,...,5.00824,5.51019,5.95725,7.04992,7.02469,6.58416,6.27058,3.85042,dcl,train.dr1.mcpm0.sa1
3,4,10.53679,9.12147,10.84621,13.92331,13.52476,10.27831,8.97459,11.57109,12.35839,...,5.85688,5.40324,6.07126,5.30651,4.27412,3.63384,3.22823,4.63123,dcl,train.dr1.mcpm0.sa1
4,5,12.96705,13.69454,14.91182,18.22292,18.45390,17.25760,17.79614,17.76387,18.99632,...,8.00151,7.58624,6.65202,7.69109,6.93683,7.03600,7.01278,8.52197,aa,train.dr1.mcpm0.sa1


In [16]:
# Dropping row.names
df = df.drop(['row.names','speaker'], axis=1)
df.head()

,x.1,x.2,x.3,x.4,x.5,x.6,x.7,x.8,x.9,x.10,...,x.248,x.249,x.250,x.251,x.252,x.253,x.254,x.255,x.256,g
0,9.85770,9.20711,9.81689,9.01692,9.05675,8.92518,11.28308,11.52980,10.79713,9.04747,...,12.94519,12.68076,11.20767,13.69394,13.72055,12.16628,12.92489,12.51195,9.75527,sh
1,13.23079,14.19189,15.34428,18.11737,19.53875,18.32726,17.34169,17.16861,19.63557,20.15212,...,8.91567,8.45714,8.77266,9.59717,8.45336,7.57730,5.38504,9.43063,8.59328,iy
2,10.81889,9.07615,9.77940,12.20135,12.59005,10.53364,8.54693,9.46049,11.96755,12.05282,...,6.49345,5.00824,5.51019,5.95725,7.04992,7.02469,6.58416,6.27058,3.85042,dcl
3,10.53679,9.12147,10.84621,13.92331,13.52476,10.27831,8.97459,11.57109,12.35839,10.47826,...,5.36506,5.85688,5.40324,6.07126,5.30651,4.27412,3.63384,3.22823,4.63123,dcl
4,12.96705,13.69454,14.91182,18.22292,18.45390,17.25760,17.79614,17.76387,18.99632,17.40394,...,7.74256,8.00151,7.58624,6.65202,7.69109,6.93683,7.03600,7.01278,8.52197,aa


In [18]:
# Exploratory Data Analysis
unique_phonemes = df['g'].unique()

print('Number of observations: {}'.format(len(df)))
print(unique_phonemes)

Number of observations: 4509
['sh' 'iy' 'dcl' 'aa' 'ao']


### Items for Neural Net
1. Break into Training & Test Sets
1. One-Hot Encode Categorical Phonemes
2. Scale Data
4. Build Network Toplogy
5. Tune weights through back-propagation
6. Iterate on time for training

In [52]:
# Training and Test Sets
from sklearn.model_selection import train_test_split

# Break in X matrix and y vector
X = df.iloc[:,:-1]
y = df.iloc[:,-1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2)
X_train.head()
print('length of X_train:',len(X_train))
print('length of X_test:',len(X_test))
print('length of y_train:',len(y_train))
print('length of y_test:',len(y_test))

length of X_train: 3607
length of X_test: 902
length of y_train: 3607
length of y_test: 902


In [48]:
# One-Hot Encode Column g corresponding to Phonemes
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

def encode(y):
    """
    Input: y prediction vector; either train or test
    Output: One-Hot Encoded Categorical array corresponding to Phonemes
    """
    encoder = LabelEncoder()
    encoded_Y = encoder.fit_transform(y)
    dummy_y = to_categorical(encoded_Y)
    return dummy_y
y_train = encode(y_train)
y_test = encode(y_test)
y_test
y_train

array([[0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1.],
       ...,
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1.]])

In [ ]:
import pandas as pd
import numpy as np
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from keras import backend


# Create Baseline model
seed = 7

def baseline_model():
  model = Sequential()
  model.add(Dense(256, input_dim=256, kernel_initializer='normal', activation='relu'))
  model.add(Dense(128,activation='relu'))
  model.add(Dense(5, kernel_initializer='normal', activation='softmax'))

  # Compile model
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

# Standardize and Pipeline
estimators=[]
estimators.append(('standardize',StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=baseline_model, epochs=100, batch_size=5,verbose=0)))
pipeline=Pipeline(estimators)

# K-Fold Cross-Validation & Cross-Validation Score
# kfold = KFold(n_splits=3, shuffle=True, random_state=seed)
# results = cross_val_score(pipeline, X, y, cv=kfold)
# print(results)

pipeline.fit(X_train,y_train)
y_pred = pipeline.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
cm = pd.DataFrame(cm)
print(classification_report(y_test, y_pred))

In [41]:
# Create Function to Scale input matrix (both train and test sets)
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier

def neural_network(X):
    model = Sequential()
    model.add(Dense(256, input_dim = 9, kernel_initializer='normal', activation = 'relu'))
    model.add(Dense(5, kernel_initializer='normal', activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Standardize Input Data & Pipeline with model
estimators = []
estimators.append(('standardize',StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=neural_network, epochs = 200, batch_size = 5, verbose = 0)))
pipeline = Pipeline(estimators)

AttributeError: module 'pandas.core.computation' has no attribute 'expressions'